# Práctico 6: Detectar comunidades

# Introducción

El objectivo de este práctico es introducir al estudiante en técnicas de detección de comunidades y solapamiento.

En la primera parte, vamos a intentar seguir lo más posible las secciones 4.3 a 4.6 del libro [SANDR].

Luego vamos a reproducir algunos de los resultados usando redes neuronales como hicimos el práctico pasado.

In [1]:
!pip install python-igraph > /dev/null
!pip install cairocffi > /dev/null

!pip freeze | grep torch #ver compatibilidad entre librerías
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html > /dev/null
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html > /dev/null
!pip install -q torch-geometric > /dev/null

Vamos a descargar algunos datasets

In [2]:
!wget -q "https://raw.githubusercontent.com/prbocca/na101_master/master/homework_06_communities/karate.graphml" -O "karate.graphml"
!wget -q "https://raw.githubusercontent.com/prbocca/na101_master/master/homework_06_communities/yeast.graphml" -O "yeast.graphml"
!wget -q "https://raw.githubusercontent.com/prbocca/na101_master/master/homework_06_communities/aidsblog.graphml" -O "aidsblog.graphml"

In [3]:
import igraph as ig
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
g_karate = ig.Graph.Read_GraphML("karate.graphml")
g_yeast = ig.Graph.Read_GraphML("yeast.graphml")
g_aidsblog = ig.Graph.Read_GraphML("aidsblog.graphml")

In [5]:
g_karate.summary()

In [6]:
g_yeast.summary()

In [7]:
g_aidsblog.summary()

#1) Cohesión en redes

Vamos a comenzar con medidas de cohesión en redes, algunas de con las cuales ya trabajamos en practicos anteriores.

Esta sección se basa en la Sección 4.3 del libro [SANDR].

##1.1) Cliques

Como habíamos mencionado hace un par de prácticos, la cantidad de cliques de una red es una forma fácil de estudiar que tan estructurado es un grafo.

Calculemos la cantidad de cliques según su tamaño.

In [8]:
from collections import Counter

#TIP: usar función cliques() para encontrar la cantidad de cliques de cada tamaño.
### START CODE HERE
### END CODE HERE
print(clique_lengths) 

Has obtenido el mismo resultado que en [SANDR]?

Ahora, calculemos la cantidad de cliques maximales.

In [9]:
### START CODE HERE
### END CODE HERE
print(maximal_cliques) 

In [10]:
# Cuál es la diferencia entre los cliques normales y los maximales?

### START CODE HERE
### END CODE HERE


##1.2) k-coreness

Una versión relajada del concepto de clique es el concepto de `k-core`. Esto consiste en un subgrafo donde todos los nodos tienen grado al menos `k` y a la vez el subgrafo es maximal (no está incluido dentro de otro grafo).

In [11]:
#cores es una lista del core al que pertence cada vértice
cores = g_karate.coreness()

Puedes generar una visualización del `4-core` correspondiente al club de karate?

In [12]:
#TIP: obtener los vértices del 4-core y crear el subgrafo 'g_karate_4core' con ellos.
### START CODE HERE
### END CODE HERE

visual_style = dict()
visual_style["bbox"] = (300, 300)
visual_style["label"] = [v["label"] for v in g_karate_4core.vs]
visual_style["vertex_color"] = "red"
ig.plot(g_karate_4core, **visual_style)

##1.3) Motifs: Contando subgrafos de orden 2 y 3

Otra variante diferente a contar cliques y sus relajaciones, consiste en contar la cantidad de subrafos de orden 2 (y sus tipos) y la cantidad de grafos de orden 3 (y sus tipos).

**Recordar** que el orden de un grafo es su cantidad de vértices.

En un grafo con 2 nodos (pareja, *dyad* en inglés), hay tres posibles estados (sin arista, 1 arista dirigida y una arista mutua).

En un grafo con 3 nodos (tríada, *triad*) hay 16 posibilidades desde el grafo sin aristas hasta el completo $K_3$.

En general a estos pequeños subgrafos se los conoce como "motifs".

Calculemos estos valores para el grafo `aidsblog`.

Primero simplifiquemos el grafo.

In [13]:
g_aidsblog = g_aidsblog.simplify()

Luego, calculemos los motifs de orden 2, con la ayuda de la función `dyad_census()`.

In [14]:
dyad_census_aidsblog = g_aidsblog.dyad_census()

print('La cantidad de parejas según tipo son:')
print(sorted(dyad_census_aidsblog.as_dict().items()))

Ahora, calcular los motifs de orden 3.

In [15]:
### START CODE HERE
### END CODE HERE

print('La cantidad de tríadas según tipo son:')
print(list(triad_census_aidsblog))
# Para interpretar los 16 tipos de motifs incluidos, ver: https://igraph.org/python/doc/api/igraph.datatypes.TriadCensus.html


##1.4) Medidas de "densidad"

Por un lado, una medida interesante en un grafo es la cantidad de aristas totales con respecto a la cantidad de aristas posible: la densidad. 

Calculemos esto para el grafo del club de karate.

In [16]:
### START CODE HERE
### END CODE HERE

print(density_karate)

Observemos que los subgrafos entorno al instructor y al administrador son notablemente más densos que el grafo global.

Usemos la `egonet` como subgrafo del entorno a un vértice. Esto es el vértice y sus vecinos directos.

In [17]:
g_karate.vs[0]["name"], g_karate.vs[33]["name"] # indices de Instructor y administrador

In [18]:
g_ego_instr = g_karate.induced_subgraph(g_karate.neighborhood(vertices=0)) #subgrafo egonet
g_ego_admin = g_karate.induced_subgraph(g_karate.neighborhood(vertices=33))
print('densidad de egonet del instructor: ', g_ego_instr.density()) #0.25
print('densidad de egonet del administrador: ', g_ego_admin.density()) #0.2091503

Otra medida de interes es la llamada "transitividad". Esta medida relacionada con el coefficiente de clustering de un grafo cuenta que proporción de subgrafos de 3 nodos (camino de largo 3), forman triangulos ($K_3$).


In [19]:
### START CODE HERE
### END CODE HERE

print(transitivity_karate)

Para la transitividad también podemos calcular una medida local por nodo, por ejemplo, para el instructor y el administrador tenemos lo siguiente.

In [20]:
g_karate.transitivity_local_undirected(vertices=[0, 33])

##1.5) Connectividad, cortes y flujos

En esta sección nos vamos a enfocar en la noción de "conexión" dentro del grafo.

Primero, observamos que el grafo de levaduras (yeast) no es conexo.

In [21]:
g_yeast.is_connected()

Al ser tan grande, una visualización rápida no es fácil de hacer...

In [22]:
visual_style = {}
visual_style["layout"] = g_yeast.layout_drl()
visual_style["bbox"] = (500, 500)
visual_style["margin"] = 10
visual_style["vertex_size"] =3
ig.plot(g_yeast, **visual_style)

Podemos calcular el tamaño de cada componente

In [23]:
Counter([g.vcount() for g in g_yeast.decompose()])

Claramente hay una componente más grande que todas las otras. Esto en general es basatante común. A este componente se le llama la componente gigante.

Calculemos el subgrafo de la componente gigante.

In [24]:
g_yeast_gc = g_yeast.decompose()[0]
print("Verificamos que nos quedemos con la más grande: N =", g_yeast_gc.vcount())

Visualizar la componente gigante.

In [25]:
### START CODE HERE
### END CODE HERE


Un efecto bastante común es el llamado "pequeño mundo" que en la prática significa que el camino promedio entre dos nodos es bastante más pequeño que el diámetro del grafo.

In [26]:
g_yeast_gc.average_path_length()

In [27]:
g_yeast_gc.diameter()

Decimos que un grafo está `k-vertices-conectado` o `k-aristas-conectado` si podemos remover `k-1` nodos (o aristas) cualesquiera y el grafo resultante es conexo. Calculemos `k` para la componente gigante del grafo de levaduras.

In [28]:
g_yeast_gc.vertex_connectivity()

In [29]:
g_yeast_gc.edge_connectivity()

Otra noción interesante es la de "vértices de corte", los cuales si se remueven, el grafo queda desconexo. 

In [30]:
cut_vertices = g_yeast_gc.cut_vertices()

In [31]:
len(cut_vertices) / g_yeast_gc.vcount() * 100 # Porcentaje de nodos que si se eliminan desconectan el grafo (son bastantes!)

#2) Particiones de grafos

Una **partición** de un grafo es simplemente una agrupación de nodos en conjuntos disjuntos (llamados **comunidades**) de forma tal que si unimos todos los conjuntos, recuperamos el grafo original.

Esto es un caso particular del problema de **detección de comunidades**, donde los nodos si pueden pertencer a más de uno de estos conjuntos.

La idea es encontrar "los mejores conjuntos de nodos" de acuerdo a alguna métrica que refleje la relación entre estos nodos (es decir, pensar en comunidades).


Esta sección se basa en las Secciones 4.4, 4.5 y 4.6 del libro [SANDR].

##2.1) Modularidad

La métrica más utilizada para determinar buenos agrupamientos se conoce como **modularidad**.

La modularidad mide que tan bien una red es particionada en
comunidades, buscando que la densidad de aristas dentro de cada comunidad sea mucho mayor a la esperada. Donde se supone que la esperada surge de un grafo aleatorio con la misma distribución de grado que el grafo original.

Cuanto mayor es la modularidad, más densidad hay dentro de las comunidades respecto a lo esperado, y por tanto mejor está realizada la partición.

Calcular todas las posibles agrupaciones de nodos y elegir la que tiene el puntaje de modularidad más alto es computacionalmente muy dificil. En su lugar, se puede utilizar un algoritmo `greedy` basado en un cluster jerárquico aglomerativo.

In [32]:
dend_k = g_karate.community_fastgreedy()
clusters_k = dend_k.as_clustering()

print(dend_k.summary())
print(clusters_k.summary())

Los tamaños de cada comunidad.

In [33]:
clusters_k.sizes()

La modularidad de la partición (cuanto mayor mejor, sabemos que  $modularidad \in [-1,1]$).

In [34]:
clusters_k.modularity

La comunidad a la que pertence cada vértice.

In [35]:
print(clusters_k.membership)

Visualizo, color según estas comunidades...

In [36]:
visual_style = dict()
visual_style["bbox"] = (400, 400)

#transformo numero de colores a paleta
id_gen = ig.datatypes.UniqueIdGenerator()
color_indices = [id_gen.add(value) for value in clusters_k.membership]
palette = ig.drawing.colors.ClusterColoringPalette(len(id_gen))
colors = [palette[index] for index in color_indices]
visual_style["vertex_color"] = colors 

ig.plot(g_karate, **visual_style)

##2.2) Partición espectral

Otra idea bastante común a la hora de calcular particiones es la de usar la descomposicón espectral del grafo.

Recordamos que para esto nos valemos de la matriz Laplaciana, $\mathbf{L}$.

Recordar: existe una fuerte relación entre los valores propios de $\mathbf{L}$ y la cantidad de componentes conexas del grafo. Donde, siempre el menor valor propio $\lambda_1 = 0$, y si tiene $n$ valores propios nulos entonces el grafo tiene $n$ componentes conexas. 

De lo anterior, sabemos que el segundo valor propio más chico $\lambda_2$ decide si el grafo es conexo o no. Y por tanto, una idea para particionar el grafo en 2 comunidades consiste en mirar el vector propio asociado a $\lambda_2$ y asumir que todos los nodos con una componenete positiva en ese vector pertenecen a una comunidad, y los que no a la otra. Este vector se conoce como el vector de `Fiedler`.
Ver justificación en el Teórico.


Primero calculo los valores y vectores propios. Y los ordeno de forma creciente.

In [37]:
L = np.array(g_karate.laplacian())

In [38]:
eig_val, eig_vec = np.linalg.eig(L)

idx = eig_val.argsort()
eig_val = eig_val[idx]
eig_vec = eig_vec[:, idx]

In [39]:
fig, ax = plt.subplots()
plt.plot(eig_val, 'bo')
plt.ylabel('Eigenvalues of Graph Laplacian')
plt.xlabel('Eigenvector index')
plt.show()

Elijo el segundo vector propio. 
Y reviso cuales de sus elementos están por arriba o abajo del cero (esto se corresponderá con la partición a la que pertenece cada vértice).

In [40]:
fiedler_vector = eig_vec[:, 1] #el segundo menor vector propio (indice comienza en cero)

In [41]:
# El color representa la facción a la que pertenece el nodo
# Los que están por arriba del cero y por debajo es lo que detecta el método de partición espectral
# Vemos que funciona bastante bien (solo falla en un punto)

fig, ax = plt.subplots()
for i in range(g_karate.vcount()):
  c = "red" if g_karate.vs['Faction'][i] == 1 else "green"
  ax.scatter(i, fiedler_vector[i], c=c)
ax.axhline(y=0, linestyle="--")
plt.ylabel('Element value of Fiedler eigenvector')
plt.xlabel('Element index (node)')
plt.show()

Visualizo, color según estas comunidades...

In [42]:
visual_style = dict()
visual_style["bbox"] = (400, 400)

membership = (fiedler_vector > 0).astype(int)

#transformo numero de colores a paleta
id_gen = ig.datatypes.UniqueIdGenerator()
color_indices = [id_gen.add(value) for value in membership]
palette = ig.drawing.colors.ClusterColoringPalette(len(id_gen))
colors = [palette[index] for index in color_indices]
visual_style["vertex_color"] = colors 

ig.plot(g_karate, **visual_style)

Todo el procedimiento anterior para encontrar el particionado espectral fue manual.

También disponemos de una función para realizar el particionado espectral,
a la cual le podemos decir la cantidad de comunidades deseadas.

In [43]:
clusters_leading_eigenvector = g_karate.community_leading_eigenvector(clusters=2)

clusters_leading_eigenvector.summary()

Lamentablemente no siempre es posible para el algortimo dividir en la cantidad de comunidades deseadas. Por suerte en este caso si pudo hacerlo.

Visualizo nuevamente. En este caso el vértice 3 queda bien asignado. Seguramente debido a pequeñas diferencias en el cálculo de ambos métodos.

In [44]:
visual_style = dict()
visual_style["bbox"] = (400, 400)

#transformo numero de colores a paleta
id_gen = ig.datatypes.UniqueIdGenerator()
color_indices = [id_gen.add(value) for value in clusters_leading_eigenvector.membership]
palette = ig.drawing.colors.ClusterColoringPalette(len(id_gen))
colors = [palette[index] for index in color_indices]
visual_style["vertex_color"] = colors 

ig.plot(g_karate, **visual_style)

##2.3) Mezcla selectiva (*assortativity mixing*)

El coeficiente de asortatividad es una medida de cuanto los nodos de una clase (asumiendo que estos tienen una clase asignada) se mezclan con nodos de otras clases. 

La forma simple de calcular esto es comprar la cantidad de aristas que hay entre clases y dentro de la misma clase contra el caso aleatorio.

La métrica $r_a$ (coeficiente de asortatividad) varía entre -1 y 1 y cuantifica esto. 
Si $r_a = 0$, esto significa que la mezcla entre clases no es diferente al caso aleatorio.
Si $r_a = 1$, esto significa que los nodos se conectan solo con otros nodos de su misma clase.

Observación: Si no hay aristas dentro de la misma clase, el coeficiente no es $-1$.

Calculemos el coeficiente para el grafo de levaduras

Trabajemos con el grafo de lavadura, que tiene 14 clases.

In [45]:
class2num = dict((x, i) for i, x in enumerate(list(set(g_yeast_gc.vs["Class"]))))

class2num #diccionario para transformar los nombres de las clases en enumerados

El coeficiente de asortatividad es:

In [46]:
g_yeast_gc.assortativity_nominal([class2num[c] for c in g_yeast_gc.vs["Class"]], directed=False)

También puede calcularse solo para una clase, la clase 'P' como ejemplo.

In [47]:
class_p = (np.array(g_yeast_gc.vs["Class"]) =='P')
print(class_p)

g_yeast_gc.assortativity_nominal(class_p, directed=False)

#3) Partición de red de políticos de USA: comparar algoritmos de detección de comunidades

Estudiaremos la red de blogs políticos de EE.UU,
con el objetivo de particionarla en las dos comunidades políticas existenes: liberales (demócratas) y conservadores (republicanos).

Los datos son de la elección política de EE.UU. en 2004, fueron recolectados por L. Adamic and N. Glance en 2005, y pueden obtenerse de la colección de Mark Newman en [link](http://www-personal.umich.edu/~mejn/netdata/).

En este ejercicio usaremos una versión no dirigida del grafo dirigido original, donde las aristas corresponden a *hiperlinks* entre blogs. La red tiene $N_v = 1490$ blogs (vértices), y se conoce la afiliación política de cada blogger (y por tanto de sus blogs), representada por un vector binario ($0$ es liberal, y $1$ es conservador) para cada vértice.

In [48]:
!wget -q "https://raw.githubusercontent.com/prbocca/na101_master/master/homework_06_communities/political.graphml" -O "political.graphml"

In [49]:
g_political = ig.Graph.Read_GraphML("political.graphml")

Reviso si el grafo es no dirigido:

In [50]:
g_political.is_directed()

Dado que el grafo se cargó como dirigido lo transformo a no dirigido.

In [51]:
g_political = g_political.as_undirected()

g_political.is_directed()

Veo los atributos de los vértices. Y en particular la afiliación partidaria (atributo `party`).

In [52]:
g_political.vs[0].attributes()

In [53]:
Counter(g_political.vs["party"]) # Hay casi la misma cantidad de cada partido.

Visualizo el grafo, según afiliación política. Se observa una clara mezcla selectiva.

In [54]:
visual_style = dict()
visual_style["bbox"] = (500, 500)
visual_style["layout"] = g_political.layout_graphopt()
visual_style["vertex_size"] = 5

#transformo numero de colores a paleta
id_gen = ig.datatypes.UniqueIdGenerator()
color_indices = [id_gen.add(value) for value in g_political.vs['party']]
palette = ig.drawing.colors.ClusterColoringPalette(len(id_gen))
colors = [palette[index] for index in color_indices]
visual_style["vertex_color"] = colors 

ig.plot(g_political, **visual_style)

Calcular el coeficiente de asortatividad para la afiliación partidaria. Y verificar que es mayor a $0.8$.

In [55]:
### START CODE HERE
### END CODE HERE


##3.1) Crear conjunto de entrenamiento y prueba

En lo que queda del práctico, le tocará comparar 3 algoritmos usados para la detección de comunidades:

* Particionado goloso rápido
* Particionado espectral
* Clasificación usando una red neuronal para grafos como hemos visto en el práctico anterior.

Primero, separaremos los vértices en dos conjuntos. Uno para entrenamiento (*train*) y otro para prueba (*test*).
Con el objetivo de comparar calidad en la detección de comunidades.

El conjunto de entrenamiento tiene solo 50 vértices, los primeros 25 y los últimos 25 según el indexado del grafo.

In [56]:
N = g_political.vcount()
train_mask = np.zeros(N, dtype=np.bool)
train_mask[:25] = 1
train_mask[-25:] = 1
print ("Son: ", N, " vértices, y para el entrenamiento se usan: ", sum(train_mask))

In [57]:
Counter(g_political.vs.select(lambda x: train_mask[x.index])["party"]) # We have a balanced train set

##3.2) Goloso Rápido

Obtengo el dendograma del particionado goloso.

In [58]:
### START CODE HERE
### END CODE HERE

dend_fastgreedy.summary()

Intento elegir un particionado en dos comunidades (buscando separar en los dos partidos políticos).

In [59]:
try:
  clusters_fastgreedy = dend_fastgreedy.as_clustering(n=2) # fails
except Exception as e:
  print(e)

Lamentablemente el dendograma no se puede partir en dos.

Voy a observar cuantas son las comunidades óptimas según este algoritmo, y buscar un método alternativo para partir en dos.

In [60]:
clusters_fastgreedy = dend_fastgreedy.as_clustering()
clusters_fastgreedy.summary()

In [61]:
Counter(clusters_fastgreedy.sizes())

Vamos a ignorar todas las pequeñas comunidades y trabajar tan solo con las 2 más grandes.
A estos nodos, vamos a intentar asignarles una de las dos clases y ver cual resulta en mayor accuracy.

**Nota**: A pesar de que no lo haremos aquí, es sencillo aplicar la maximización de modularidad manualmente. El procedimiento es similar al realizado manualmente para el particionado espectral, pero aquí debe maximizarse la matriz de modularidad. Ver Teórico.

In [62]:
df_fastgreedy = pd.DataFrame(data=clusters_fastgreedy.membership)
df_fastgreedy.columns = ["community"]
df_fastgreedy["real_party"] = g_political.vs["party"]
df_fastgreedy["real_party"] = df_fastgreedy["real_party"].astype(int)

display(df_fastgreedy.head())

Vamos a eliminar las comunidades que tienen menos de 100 (forma fácil de quedarnos con las dos más grandes)

In [63]:
community_count = (
    df_fastgreedy["community"]
    .value_counts()
    .where(lambda x: x > 100)
    .dropna()
    .index
)
df_fastgreedy = df_fastgreedy[
                              df_fastgreedy["community"]
                              .isin(community_count)
                              ] # Eliminando las pequeñas comunidades

display(df_fastgreedy.head())

Counter(df_fastgreedy["community"])

Ahora evaluaremos la exactitud de nuestro método.
Para esto tenemos que comparar la afilización partidaria (`real_party`) con las dos comunidades encontradas por el algoritmo.

No sabemos que comunidad corresponde a cada partido, por tanto comparamos ambos mapeos posibles.

In [64]:
df_fastgreedy["assignament_1"] = df_fastgreedy["community"].map({0: 0, 3: 1})
(df_fastgreedy["real_party"] == df_fastgreedy["assignament_1"]).sum() / df_fastgreedy.shape[0]

Con el primer mapeo entre comunidad y partido político tuvimos una accuracy de casi el 95%.

In [65]:
df_fastgreedy["assignament_2"] = df_fastgreedy["community"].map({0: 1, 3: 0})
(df_fastgreedy["real_party"] == df_fastgreedy["assignament_2"]).sum() / df_fastgreedy.shape[0]

Con el opuesto tan solo un 4%. Claramente el mapeo correcto es el primero y la accuracy es del 95%.

Usando el estilo anterior, visualizo donde me equivoqué n la asignación.

Debería ser mayor en los vértices menos conectados de la periferia, pero eso no se observa en el dibujo.

In [66]:
#usando la visualizacion anterior, veo donde me equivoqué
correct = (df_fastgreedy["real_party"] == df_fastgreedy["assignament_1"]) #vertices bien clasificados
visual_style["vertex_shape"] = ["circle" if c else "square" for c in correct] 
visual_style["vertex_size"] =  [5 if c else 10 for c in correct] 

ig.plot(g_political, **visual_style)

##3.3) Particionado Espectral

Vamos a repetir el proceso una vez más pero para el algoritmo de particionado 
espectral. 

Usemos la función `community_leading_eigenvector()` para calcular el particionado espectral con dos comunidades.

In [67]:
### START CODE HERE
### END CODE HERE

clusters_leading_eigenvector.summary()

Observamos que no fue posible para el algoritmo particionar en solo dos comunidades.
Aquí tenemos dos opciones: 
* aplicar el método sencillo usado cuándo no pudimos particionar en dos comunidades el método goloso
* implementar el particionado espectral manualmente con la cantidad de comunidades deseadas (2).

Por sencillez usaremos la primera opción:

In [68]:
#repetimos la deteccion de comunidades buscando las comunidades óptimas
clusters_leading_eigenvector = g_political.community_leading_eigenvector()

clusters_leading_eigenvector.summary()

In [69]:
Counter(clusters_leading_eigenvector.sizes())

Repetimos en este algoritmo la evaluación de calidad realizada para el algoritmo goloso.

In [70]:
df_eigen = pd.DataFrame(clusters_leading_eigenvector.membership)
df_eigen.columns = ["community"]
df_eigen["real_party"] = g_political.vs["party"]
df_eigen["real_party"] = df_eigen["real_party"].astype(int)

display(df_eigen.head())

In [71]:
community_count = (
    df_eigen["community"]
    .value_counts()
    .where(lambda x: x > 100)
    .dropna()
    .index
)
df_eigen = df_eigen[
    df_eigen["community"]
    .isin(community_count)
] # Eliminando las pequeñas comunidades

display(df_eigen.head())

Counter(df_eigen["community"])

In [72]:
df_eigen["assignament_1"] = df_eigen["community"].map({0: 0, 268: 1})
(df_eigen["real_party"] == df_eigen["assignament_1"]).sum() / df_eigen.shape[0]

In [73]:
df_eigen["assignament_2"] = df_eigen["community"].map({0: 1, 268: 0})
(df_eigen["real_party"] == df_eigen["assignament_2"]).sum() / df_eigen.shape[0]

Otra vez, logramos una accuracy cerca del 94%!

In [74]:
#usando la visualizacion anterior, veo donde me equivoqué
correct = (df_eigen["real_party"] == df_eigen["assignament_1"]) #vertices bien clasificados
visual_style["vertex_shape"] = ["circle" if c else "square" for c in correct] 
visual_style["vertex_size"] =  [5 if c else 10 for c in correct] 

ig.plot(g_political, **visual_style)

##3.4) Red neuronal para grafos

In [75]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader

Debes implementar un cargador de grafos de Pytorch así como la clase `GCN`, tal como lo hicimos en el práctico anterior para clasificar vértices.

Recuerda usar la mascara que definimos antes.

In [76]:
#TIP los datos deben guardarse en `data`, y la clase debe llamarse `GCN()`
### START CODE HERE
### END CODE HERE


Aquí vamos a entrenar el modelo

In [77]:
model = GCN()
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Define optimizer.

def train(data):
    optimizer.zero_grad()  # Clear gradients.
    out, h = model(data.x, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss, h

for epoch in range(1000):
    loss, h = train(data)

Pasamos una vez más el modelo por la red para encontrar la clasificación en 2 clases y finalmente calculamos la exactitud.

In [78]:
out, h = model(data.x, data.edge_index)
predicted_classes = out.detach().numpy().argmax(axis=1)

In [82]:
correct = (predicted_classes == g_political.vs["party"]) #vertices bien clasificados
correct.sum() / N

Con una arquitectura super simple usando redes neuronales alcanzamos una accuracy del 81%.

In [81]:
#usando la visualizacion anterior, veo donde me equivoqué
visual_style["vertex_shape"] = ["circle" if c else "square" for c in correct] 
visual_style["vertex_size"] =  [5 if c else 10 for c in correct] 

ig.plot(g_political, **visual_style)